Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p0">Part 0</a>: Kaggle Competition
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

*Note:* Each time we call the pipeline during grid search, each component is fit again. The vectorizer (tf-idf) is transforming our entire vocabulary during each cross-validation fold. That transformation adds significant run time to our grid search. There *might* be interactions between the vectorizer and our classifier, so we estimate their performance together in the code below. However, if your goal is to reduce run time. Train your vectorizer separately (ie out of the grid-searched pipeline). 

In [63]:
# Import Statements
from sklearn.pipeline import Pipeline # we will be using this in a slightly different way than we have before
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
import ftfy

In [2]:
# Dataset
from sklearn.datasets import fetch_20newsgroups # grabs data

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [ ]:
#print out data sample

In [3]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2)) # these are parameters that we want to hard-set
rfc = RandomForestClassifier()

In [4]:
# Define the Pipeline

# remember -- this is a class instance. the list of items are vectorizer & classifier that we're putting in the instance
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect), # include name & then class instance of that item
                 # Classifier
                 ('clf', rfc) # could actually instantiate these inside the pipeline, but easier to read this way
                ])

# The pipeline puts together a bunch fit then transform,fit then predict. 

In [5]:
# use the name that you called it in the pipeline, and then __parameter
parameters = {
    'vect__max_df': ( 0.75, 1.0), 
    'vect__min_df': (.02, .05), # must be included in at least this percentage to be included
    'vect__max_features': (500,1000), # max # features pipeline will extract -- you want the top x most frequent features
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   33.7s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  2.2min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [6]:
grid_search.best_score_

0.8856476079346558

In [7]:
grid_search.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

array([1, 1])

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [59]:
import pandas as pd
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [10]:
train.head()

# To Do: clean data
# try new classifier

,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [109]:
# Remove html remnants from descriptions
train['clean_description'] = train['description'].str.replace(r'\xa0', '').str.strip()

In [122]:
test['clean_description'] = test['description'].str.replace(r'\xa0', '').str.strip()

### Define Pipeline Components

In [9]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier() #clf = classifier

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [20]:
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (1000, None),
    'clf__max_depth':(5,10,15,20),
    'clf__n_estimators': (5, 10)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 320 out of 320 | elapsed:  8.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (1000, None), 'clf__max_depth': (5, 10, 15, 20), 'clf__n_estimators': (5, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [21]:
grid_search.best_score_

0.7225348666503548

In [22]:
grid_search.best_params_

{'clf__max_depth': 20,
 'clf__n_estimators': 10,
 'vect__max_df': 0.75,
 'vect__max_features': 1000,
 'vect__min_df': 0.02}

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this compeition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [13]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [15]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [16]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [17]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission1.csv', index=False)

## Challenge

You're trying to achieve 90% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

In [ ]:
# works nicely with dimensionality reduction for text
# originially created for information retrieval
# truncated singular value decomposition (the main theory this relies on)
# it's roughly comparable to PCA (that's all we need to know)
# select the number of components

# this is becoming less important with modern word embedding models
# because word embedding vectors already have fixed length -- so don't combine these two 

# REMEMBER : if using tfidf or count vectorizer, it's important to use dimensionality reduction

In [23]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, # Just here for demo. 
                   algorithm='randomized',
                   n_iter=10)

# create one pipeline (lsi) that has vectorizer & singular value decomp
# then separate from our LSI and classifier

In [24]:
params = { 
    'lsi__svd__n_components': [10,100,250], # this is how you call on subcomponents of pipelines
    'lsi__vect__max_df':[.9, .95, 1.0],
    'clf__n_estimators':[5,10,20]
}

In [25]:
# mashing together pipelines like so:
# why can we do this? expecting a lot of transform statements with final predict statement
# so 
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)]) #


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [26]:
print(pipe)

Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])


In [27]:
# Fit
grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  9.8min
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed: 25.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'lsi__svd__n_components': [10, 100, 250], 'lsi__vect__max_df': [0.9, 0.95, 1.0], 'clf__n_estimators': [5, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [28]:
grid_search.best_score_

0.9136522753792299

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [51]:
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
import xgboost as xgb
from scipy.stats import randint, uniform

In [36]:

svd = TruncatedSVD(n_components=100, algorithm='randomized', n_iter=10)
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = xgb.XGBClassifier() #clf = classifier

lsi = Pipeline([('vect', vect), ('svd', svd)])
pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [43]:
parameters = {
    'lsi__svd__n_components': randint(10, 250),
    'lsi__vect__max_df': uniform(0.75, 1.0),
    'clf__max_depth': randint(3, 100),
    'clf__learning_rate': uniform(0, 1)
}

random_search = RandomizedSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
random_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 20.0min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 22.4min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1))]),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'lsi__svd__n_components': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1125bcf28>, 'lsi__vect__max_df': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1125c2a20>, 'clf__max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x119ead0b8>, 'clf__learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x119eada20>},
          pre_dispatch='2*n_jobs'

In [44]:
random_search.best_score_

0.7355028137998532

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [45]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [46]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [47]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [48]:
subNumber = 0

In [49]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [52]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [20]:
doc = nlp("Two bananas in pyjamas")

In [21]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [57]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [58]:
X = get_word_vectors(train['description'])

len(X) == len(train['description'])

True

857

In [26]:
rfc.fit(X, data.target)

/Users/jonathansokoll/anaconda3/envs/U4-S1-NLP-DS6/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [27]:
rfc.score(X, data.target)

0.9883313885647608

## Follow Along

In [54]:
# Apply to your Dataset

## picked back up here

from sklearn.ensemble import GradientBoostingClassifier

param_dist = {
    'max_depth' : randint(3, 10),
    'min_samples_leaf': randint(2, 15)
}

In [55]:
clf = GradientBoostingClassifier()
search = RandomizedSearchCV(clf, param_dist, cv=5, n_iter=10, n_jobs=-1, verbose=1)
search.fit(X, )

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

In [114]:
X = get_word_vectors(train['clean_description'])
len(X) == len(train['clean_description'])

True

In [118]:
svd = TruncatedSVD()
# vect = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
clf = xgb.XGBClassifier(random_state=7)

# lsi = Pipeline([('vect', vect), ('svd', svd)])
pipe = Pipeline([('svd', svd), ('clf', clf)])

parameters = {
    'svd__n_components': randint(10, 250),
    #'lsi__vect__max_df': uniform(0.75, 1.0),
    #'lsi__vect__max_features': (500, 1000, None),
    'clf__n_estimators': randint(500, 1000),
    'clf__learning_rate': uniform(.1, .5),
    'clf__max_depth': randint(10, 100)
}

search = RandomizedSearchCV(pipe, parameters, cv=3, n_jobs=-1, verbose=1)
search.fit(X, train['ratingCategory'])

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 19.6min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('svd', TruncatedSVD(algorithm='randomized', n_components=2, n_iter=5,
       random_state=None, tol=0.0)), ('clf', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_dept...
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1))]),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'svd__n_components': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1528f6550>, 'clf__n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x1528f63c8>, 'clf__learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x11ca937f0>, 'clf__max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x152969c50>},
          pre_dispatch='2*n_jobs', ran

In [119]:
print(search.best_score_)
search.best_params_

0.733790066063127


{'clf__learning_rate': 0.2917270158367985,
 'clf__max_depth': 96,
 'clf__n_estimators': 787,
 'svd__n_components': 71}

In [124]:
# Predictions on test sample
Y = get_word_vectors(test['clean_description'])

pred = search.predict(Y)
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [125]:
subNumber += 1
submission.to_csv(f'./data/submission{subNumber}.csv', index=False)


# Review

To review this module: 
* Continue working on the Kaggle comeptition
* Find another text classification task to work on